# X-ray CT using the Core Imaging Library (CIL) and UQ through the CUQIpy-CIL plugin

The Core Imaging Library (https://www.ccpi.ac.uk/cil) is an open-source python library for processing and reconstruction of CT data (and other inverse problems).

The CUQIpy-CIL plugin wraps CIL tools into CUQIpy, so that CUQIpy can be used to carry out UQ analysis on CT problems.

This notebook first demonstrates how to use the test problems provided by the CUQIpy-CIL plugin and set up and run a CUQIpy UQ analysis.

Next a real X-ray CT data set is loaded in, analyzed and reconstructed using CIL. 

Finally the real data case is set up using the CUQIpy-CIL plugin for UQ analysis.

### Prerequisites
Make sure you have the latest version of CIL and CUQIpy-CIL installed. See install instructions [here](https://github.com/CUQI-DTU/CUQIpy-CIL)

The notebook assumes a general familiarity with CT.

## 1. CT test problems in the CUQIpy-CIL plugin

We first load the tools we need, including the CUQIpy-CIL plugin `cuqipy_cil`:

In [ ]:
import cuqi
import cuqipy_cil
import numpy as np
import matplotlib.pyplot as plt

# For the moment, we stick to CPU only mode using astra
cuqipy_cil.config.PROJECTION_BACKEND = "astra"
cuqipy_cil.config.PROJECTION_BACKEND_DEVICE = "cpu"

Load a CT forward model and data from testproblem, which can be configured. A fan beam test problem is also available.

We stick to a fairly small image size to reduce the computational time for this demo assuming most users will be running this on a laptop with no GPU.

In [ ]:
A, y_data, info = cuqipy_cil.testproblem.ParallelBeam2DProblem.get_components(
    im_size=(90, 90),
    det_count=128,
    angles=np.linspace(0, np.pi, 90),
    phantom="shepp-logan"
)

We extract and display the exact solution image, to be reconstructed:

In [ ]:
x_exact = info.exactSolution
x_exact.plot()

The data is a sinogram, which we can plot:

In [ ]:
y_data.plot()

Instead of loading the premade noisy sinogram from the test problem we could also have generated it ourselves by forward projecting it using the forward model (and adding noise):

In [ ]:
(A@x_exact).plot()

# 2. Setting up and solving a Bayesian inverse problem

As in previous notebooks we can set up a Bayesian model, assuming a Gaussian prior for the image and additive Gaussian noise: 

In [ ]:
x = cuqi.distribution.GaussianCov(np.zeros(A.domain_dim), 1) # x ~ N(0, 1)
y = cuqi.distribution.GaussianCov(A@x, 0.05**2)              # y ~ N(Ax, 0.05^2)

From the two distributions we can either set up a JointDistribution, condition on the data and sample the posterior, or we can use the higher-level interface BayesianProblem which does all of that for us, including selecting a suitable sampler:

In [ ]:
BP = cuqi.problem.BayesianProblem(y, x)
BP.set_data(y=y_data)

Sample from the posterior (this will take a few minutes depending on your hardware):

In [ ]:
samples = BP.sample_posterior(200)

Analyze the samples:

In [ ]:
info.exactSolution.plot(); plt.title("Exact solution"); plt.show()
y_data.plot(); plt.title("Data"); plt.show()
samples.plot_mean(); plt.title("Posterior mean"); plt.show()
samples.plot_std(); plt.title("Posterior standard deviation"); plt.show()

Try instead of a Gaussian prior with an edge-preserving Cauchy difference prior:

In [ ]:
x = cuqi.distribution.Cauchy_diff(location=np.zeros(A.domain_dim), scale=0.01, physical_dim=2)
y = cuqi.distribution.GaussianCov(A@x, 0.05**2)              # y ~ N(Ax, 0.05^2)

In [ ]:
BP_C = cuqi.problem.BayesianProblem(y, x)
BP_C.set_data(y=y_data)

Sampling from this posterior may take a bit longer.

In [ ]:
samples_C = BP_C.sample_posterior(200)

In [ ]:
samples_C.plot_mean(); plt.title("Posterior mean"); plt.show()
samples_C.plot_std(); plt.title("Posterior standard deviation"); plt.show()

# 3. Reconstructing a real X-ray CT dataset using CIL

The last part of this notebook focuses on a real X-ray CT data set. The data set is of a Lotus root and is provided by the Finnish Inverse Problems Society. We will first show how to load in and explore this data and set it up for reconstruction using CIL (without CUQIpy). Afterwards it will be shown how to wrap this into CUQIpy and then the task is to carry out some UQ analysis for this data using CUQIpy.

We first load the tools we are going to need:

In [ ]:
import scipy
import numpy as np
from cil.framework import AcquisitionData, AcquisitionGeometry
from cil.utilities.display import show2D, show_geometry
from cil.recon import FDK
from cil.plugins.astra.processors import FBP
from cil.processors import Binner
from cil.plugins.astra import ProjectionOperator
from cil.optimisation.algorithms import CGLS

Go to https://zenodo.org/record/1254204/ and download the file sinogram.mat to the same directory as this notebook. This is an X-ray data set of a Lotus root, see https://arxiv.org/pdf/1609.07299.pdf for details.

The data can be loaded and cropped according to instructions in the original code accompanying the data:

In [ ]:
sinogram = scipy.io.loadmat('sinogram.mat')['sinogram'].T
sinogram = sinogram[0:360, 0:2221]

In [ ]:
sinogram.shape

The parameters needed to specify the scan geometry are provided in the orignal code `Lotus_FBP.m`

In [ ]:
num_angles = sinogram.shape[0]
num_dets = sinogram.shape[1]
source_center = 54
source_detector = 63
det_pixel_size = 12/2240

We specify the projection angles:

In [ ]:
angles = np.linspace(0, 360, num_angles, endpoint=False)

We create using CIL commands the `AcquisitionGeometry` representing the scan geometry:

In [ ]:
ag_full = (
    AcquisitionGeometry.create_Cone2D(
        source_position=[0, -source_center],
        detector_position=[0, source_detector - source_center],
    )
    .set_panel(num_pixels=num_dets, pixel_size=det_pixel_size)
    .set_angles(angles=-angles, angle_unit="degree")
)

And put it in the CIL data structure `AcquisitionData`:

In [ ]:
data_full = AcquisitionData(sinogram, geometry=ag_full)

We can take a look at the data by the CIL plot show2D:

In [ ]:
show2D(data_full)

We can take a look at the scan geometry using a CIL plot:

In [ ]:
show_geometry(ag_full)

Before proceeding we provide the opportunity to downsample the data to make experiments run faster. Choose an integer factor for the angles and the horizontal detector pixel dimensions:

In [ ]:
ds_angle = 1
ds_horizontal = 10

In [ ]:
data = Binner(roi={'angle':(0,360,ds_angle),'horizontal':(0,2221,ds_horizontal)})(data_full)
print(data)

We extract the acquisition geometry of the downsampled data and can specify the corresponding geometry of the image to be reconstructed onto, here the default choice:

In [ ]:
ag = data.geometry
print(ag)

In [ ]:
ig = ag.get_ImageGeometry()
print(ig)

Now we can use CIL reconstruction tools. We specify the CIL forward model and use the CGLS algorithm for a basic early-stopping least squares reconstruction:

In [ ]:
AP = ProjectionOperator(ig, ag, device='cpu')

In [ ]:
cgls = CGLS(operator=AP, data=data, tolerance=-1, max_iteration=1000)

In [ ]:
cgls.run(30)

We can show the reconstruction, which is held in the reconstruction algorithm's `solution` attribute, using the CIL `show2D` plot:

In [ ]:
show2D(cgls.solution)

# 4. Wrapping the real data into CUQIpy

The previous section demonstrated how to reconstruct the data using CIL. This section explores how to run UQ on it using CUQIpy. We need a few CUQIpy tools:

In [ ]:
from cuqipy_cil.model import CILModel
from cuqi.array import CUQIarray

The CIL acquisition and image geometry fully specify the imaging setup and we can create a CUQIpy CIL forward model directly from these:

In [ ]:
model = CILModel(ag, ig)
print(model)

Instead of using CIL data structures for the sinogram data we need to use the CUQIpy data structure CUQIarray, where we provide its geometry from the model's range geometry:

In [ ]:
data_cuqi = CUQIarray(data.as_array().ravel(), is_par=True, geometry=model.range_geometry)
data_cuqi

We now have all we need to start using CUQIpy for UQ analyzing the data. To demonstrate this, we can see if the CUQIpy plotting is available for the data_cuqi:

In [ ]:
data_cuqi.plot()

As we hoped, this works and gives an Image2D type plot of the sinogram.

We can check that the CUQIpy forward model is working, for example by applying its adjoint operation (corresponding to backprojection) onto the CUQIarray holding the sinogram:

In [ ]:
data_backprojected = model.T@data_cuqi
data_backprojected

And plot the results:

In [ ]:
data_backprojected.plot()

As expected a plain backprojection of the sinogram gives a blurry image in the reconstruction domain.

## 5. Open-ended exploration

With the tools in place, you can now explore UQ analysis of the data set, by use of the CUQIpy tools you have met so far.

### Ideas for exploration:

(Can be either for the test problem or the real data)

1. Set up a few-projection case, for example 40 projections (over 180 degrees for parallel-beam or 360 degrees for fan-beam). Explore different priors.
2. Set up a limited angle case, for example instead of full 180-degree data, use 135, 90 or even 45 degree data, and explore different the effect of different priors on the solution and its uncertainty.
3. Try with a smooth phantom instead of a piecewise constant (test problem only)
4. Infer noise level and/or prior precision through Gibbs.

In [ ]:
# Your code here



